In [1]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt

In [2]:
def GetRMSE(dlc_x, dlc_y, manual_x, manual_y):
    from scipy.spatial.distance import pdist

    er = []
    for idx in range(len(dlc_x)):
        X = np.asarray([[manual_x[idx], manual_y[idx]], 
                         [dlc_x[idx], dlc_y[idx]]])
        temp_er = pdist(X,metric = 'euclidean')
        er.append(temp_er[0])
    er = np.asarray(er)
    return(er)

In [3]:
outpath = '../dataFolders/Final Tracking xypts/filteredUsingThreshold/'

In [4]:
datalist = glob.glob('../dataFolders/Tracking Errors/version2-usingSuccessiveFrames/**.csv', recursive=True)

In [5]:
namelist = []
for data in datalist:
    _, tail = os.path.split(data)
    namelist.append(tail[:-4])

## version 0

In [6]:
# calculate the threshold to use as true data cutoff

uber_error = np.asarray([np.nan])
for data in datalist:
    file = pd.read_csv(data)
    error = file.loc[:, 'rmse'].values
#     error = file.loc[:,'rmse'].values
    uber_error = np.concatenate([uber_error, error])
uber_error = uber_error[1:]
len(uber_error)

10308

In [7]:
print(data)
file.head()

../dataFolders/Tracking Errors\c-3_m6_last.csv


,level_0,index,manual_x,manual_y,dlc_x,dlc_y,dlc_likelihood,rmse,zscore
0,0,0,303.878213,467.202655,301.871070,446.815884,0.999995,20.485338,2.514855
1,1,1,299.119339,466.338938,300.806893,442.983627,1.000000,23.416200,3.090140
2,2,2,299.119339,456.838053,298.052695,441.044231,1.000000,15.829799,1.601041
3,3,3,295.946756,449.064602,294.531591,438.369887,1.000000,10.787939,0.611397
4,4,4,299.119339,457.701770,295.734209,437.002952,1.000000,20.973797,2.610732


In [8]:
df = pd.DataFrame(data = uber_error, columns = ['rmse'])
df['zscore'] = (df.rmse - df.rmse.mean())/df.rmse.std(ddof = 0)

In [9]:
print('mean: %0.2f,' %df.rmse.mean(), 'median: %0.2f,' %df.rmse.median())
print('0.5std: %0.2f,' %(df.rmse.mean() + 0.5*df.rmse.std()))

mean: 14.68, median: 6.61,
0.5std: 26.61,


In [10]:
# save results as a parameters file
Parameters_DICT = {
    'mean': df.rmse.mean(),
    'median' : df.rmse.median() ,
    'std': df.rmse.std(),
    'threshold used': '0.5 std',
    'threshold value' : df.rmse.mean() + 0.5*df.rmse.std()
}

f = open(outpath + "parameters_for_threshold.txt","w")
f.write( str(Parameters_DICT) )
f.close()

In [11]:
cutoff_thresh = df.rmse.mean() + 0.5*df.rmse.std()

for data, name in zip(datalist, namelist):
    file = pd.read_csv(data)
    x = file.dlc_x.values
    y = file.dlc_y.values
    
    # get rmse values for subsequent frames
    rmse = GetRMSE(x[1:], y[1:], x[:-1], y[:-1])
    
    filtered_x = np.copy(x[1:])
    filtered_y = np.copy(y[1:])
    filtered_x[rmse > cutoff_thresh] = np.nan
    filtered_y[rmse > cutoff_thresh] = np.nan

    new_xypts = pd.DataFrame({'tracked_x': filtered_x, 'tracked_y': filtered_y, 'rmse': rmse})
    new_xypts.to_csv(outpath + '/' + name + '.csv')

## version 1 - using successive frames to figure out cutoff

In [19]:
cutoff_thresh = [24, 20]

for data, name in zip(datalist, namelist):
    file = pd.read_csv(data)
    x = file.dlc_x.values
    y = file.dlc_y.values
    
    # get rmse values for subsequent frames
    rmse = GetRMSE(x[1:], y[1:], x[:-1], y[:-1])
    
    filtered_x = np.copy(x[1:])
    filtered_y = np.copy(y[1:])
    for cutoff in cutoff_thresh:
            filtered_x[rmse > cutoff] = np.nan
            filtered_y[rmse > cutoff] = np.nan
            
            new_path = '../dataFolders/FinalTracking_xypts/filteredUsingThreshold/v1/cutoff' + str(cutoff)
            
            try:
                os.mkdir(new_path)
            except OSError:
                print('oops')
            
            new_xypts = pd.DataFrame({'tracked_x': filtered_x, 'tracked_y': filtered_y, 'rmse': rmse})
            new_xypts.to_csv(new_path + '/' + name + '.csv')

oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops
oops


## plot and test if things look right!

In [ ]:
[name for name in namelist if 'c-1_m13' in name]

In [ ]:
# fig = plt.subplots(figsize = (10,6))
plt.plot(x, 'o')
plt.plot(np.hstack([np.nan, tracked_x]), 'o')
plt.show()

plt.plot(y, 'o')
plt.plot(np.hstack([np.nan, tracked_y]), 'o')
plt.show()

# plt.plot(x[np.hstack([False, np.isnan(tracked_x)])], 'or')

In [ ]:
circle_params = pd.read_csv('../dataFolders/circle_parameters.csv')
circle_params.head()

In [ ]:
mname = [name for name in circle_params.name if name in data][0]

cent_x = circle_params.loc[circle_params.name == mname, 'circ_x'].values
cent_y = circle_params.loc[circle_params.name == mname, 'circ_y'].values
circ_radius = circle_params.loc[circle_params.name == mname, 'circ_radii'].values

centered_x_dlc = file.dlc_x - cent_x
centered_y_dlc = file.dlc_y - cent_y
r_dlc = np.linalg.norm([centered_x_dlc, centered_y_dlc], axis = 0)
radius_dlc = r_dlc/circ_radius

centered_x = tracked_x - cent_x
centered_y = tracked_y - cent_y
r = np.linalg.norm([centered_x, centered_y], axis = 0)
radius = r/circ_radius


centered_x_manual = file.manual_x - cent_x
centered_y_manual = file.manual_y - cent_y
r_manual = np.linalg.norm([centered_x_manual, centered_y_manual], axis = 0)
radius_manual = r_manual/circ_radius

In [ ]:
fig = plt.subplots(figsize = (20,6))
# plt.plot(radius_dlc)
plt.plot(radius[200:500])
plt.plot(radius_manual[201:501])